In [94]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import argrelextrema
import numpy as np
from datetime import datetime, timedelta
from pandas.core.common import SettingWithCopyWarning
import warnings

In [95]:
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [96]:
df = pd.read_csv('../data_file/isolate_merge_df.csv')

In [207]:
def find_holiday(df, i):
    building_df = df[df['building_number'] == i]
    building_df.set_index('date_time', inplace=True)
    building_df_daily_max = building_df.resample('D').max()
    # 평균
    avg_pwr = (building_df_daily_max['power_consumption'].max() + building_df_daily_max['power_consumption'].min())/2
    
    # local minima 찾기
    comparator = np.less
    local_minima_indices = argrelextrema(building_df_daily_max['power_consumption'].values, comparator)
    local_minima = building_df_daily_max['power_consumption'].iloc[local_minima_indices]
    local_minima = local_minima[local_minima < avg_pwr]
    #print(local_minima)
    
    building_df['holiday'] = 0
    for i in local_minima.index:
        building_df.loc[building_df.index[(building_df.index >= i) & (building_df.index  < (i + timedelta(days=1)))], 'holiday'] =1
    
    df[df['building_number'] == i]['holiday'] = building_df['holiday']
    return df
    
    

In [208]:
for i in range(37, 43):
    df = find_holiday(df, i)

In [ ]:
df.to_csv('../datafix_holiday_feature.csv')